In [1]:
import pandas as pd

In [2]:
# Read in data
payments = pd.read_csv("../Resources/payments.csv")

## 3) Devise a method of generating a table of all Accounts with their cumulative payments to date as of any arbitrary timestamp in the past.

In [3]:
# Preview dataframe
payments.head()

,id,effective_when,account_id,currency,amount
0,1,2019-12-26 06:00:42.571392,1,KES,950
1,2,2019-12-26 06:00:42.571392,1,KES,950
2,3,2019-11-06 05:49:39.571392,2,KES,950
3,4,2019-11-06 05:49:39.571392,2,KES,950
4,5,2019-11-11 06:43:34.571392,2,KES,415


In [ ]:
# Check for unique currencies
payments.currency.unique()

In [3]:
# Set currency
currency = payments.currency[0]

In [10]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-01-01


In [11]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)
print(timestamp_dt)
print(type(timestamp_dt))

2020-01-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
# Check data type of date in 'effective_when'
type(payments['effective_when'][0])

In [12]:
# Change data type from string to datetime object
payments['effective_when'] = pd.to_datetime(payments['effective_when'])

In [ ]:
# Check data type of date in 'effective_when'
type(payments['effective_when'][0])

In [13]:
# Create limited dataframe starting at indicated timestamp 
payments_date_limited = payments.loc[payments['effective_when'] >= timestamp_dt]
payments_date_limited.head()

,id,effective_when,account_id,currency,amount
12,13,2020-01-05 09:25:19.571392,2,KES,368
13,14,2020-01-12 11:43:07.571392,2,KES,380
14,15,2020-01-19 14:23:11.571392,2,KES,422
15,16,2020-01-27 02:37:47.571392,2,KES,388
16,17,2020-02-03 20:07:42.571392,2,KES,422


In [14]:
# Group by account_id and count number of records
num_payments = payments_date_limited.groupby('account_id').count()

# Flatten dataframe
num_payments = num_payments.rename_axis(None, axis=1).reset_index()

# Drop columns
num_payments = num_payments.drop(columns=['effective_when','currency','amount'])

# Rename column
cumulative_payments = num_payments.rename(columns={"id":"total_num_payments"})
cumulative_payments.head()

,account_id,total_num_payments
0,2,8
1,3,7
2,4,8
3,5,5
4,6,8


In [15]:
# Group by account_id and sum payment amounts
sum_payments = payments_date_limited.groupby('account_id').sum()

# Flatten dataframe
sum_payments = sum_payments.rename_axis(None, axis=1).reset_index()

# Select one column
sum_payments = sum_payments['amount']

# Add series to cumulative_payments dataframe
cumulative_payments[f'total_sum_payments_{currency}'] = sum_payments

print(f"CUMULATIVE PAYMENTS BY ACCOUNT SINCE: {timestamp_str[:10]}")
cumulative_payments.head()

CUMULATIVE PAYMENTS BY ACCOUNT SINCE: 2020-01-01


,account_id,total_num_payments,total_sum_payments_KES
0,2,8,3102
1,3,7,3213
2,4,8,4599
3,5,5,2386
4,6,8,2360


In [23]:
# Export as csv file
cumulative_payments.to_csv(f"../Output/cumulative_payments_by_account_since_{timestamp_str[:10]}.csv", index=False)